In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
import numpy as np
from tqdm import tqdm
import json
"""
File format:
<pred label> <true label>
101001000011111....
"""

N: int = 50000
PATH:str = 'trainset_mnist-net_256x6.onnx'
ONLY_CORRECT_PRED = True #whether we should extract pattern from wrong predictions or not


all_patterns = {}

for idx in tqdm(range(N)):
    with open(os.path.join(PATH, "reluBitmap_{}.txt").format(idx)) as f:
        raw = f.readlines()
        pred_label, true_label = raw[0].strip().split()
        if ONLY_CORRECT_PRED:
            if pred_label != true_label: continue
        bitmap = [*raw[1].strip()] #convert the string "01000..." to the list ["0", "1", "0", "0", ...]
        bitmap = [int(b) for b in bitmap] #convert the list of string to the list of int
        if pred_label in all_patterns:
            all_patterns[pred_label].append(bitmap)
        else:
            all_patterns[pred_label] = []
            all_patterns[pred_label].append(bitmap)

for k in all_patterns.keys():
    all_patterns[k] = np.array(all_patterns[k])

    print(k, all_patterns[k].shape)



100%|██████████| 50000/50000 [00:17<00:00, 2841.18it/s]


5 (4302, 1536)
0 (4887, 1536)
4 (4797, 1536)
1 (5627, 1536)
9 (4866, 1536)
2 (4936, 1536)
3 (4902, 1536)
6 (4895, 1536)
7 (4974, 1536)
8 (4678, 1536)


In [2]:
DELTA = 0
stable_relu_patterns = {}
for k in all_patterns.keys():
    print(k)
    sum = all_patterns[k].sum(axis = 0)
    n_sample = all_patterns[k].shape[0]
    print("n_sample", n_sample)
    neg_relus = np.where(sum <= n_sample*DELTA)
    pos_relus = np.where(sum >= n_sample*(1-DELTA))
    stable_idx = np.concatenate([neg_relus, pos_relus], axis= 1).squeeze()
    print(stable_idx)
    stable_idx = np.sort(stable_idx)
    
    stable_relu_patterns[k] = {"stable_idx": stable_idx.tolist(), 
                               "val": [int(sum[idx]) for idx in stable_idx]}

with open("datasets/MNIST/MNIST_256x6_relu_patterns_{}.json".format(DELTA), "w") as f:
    json.dump(stable_relu_patterns, f, indent=2)


5
n_sample 4302
[   1   11   20   27   28   30   67   89   93   99  106  107  108  123
  150  154  161  176  185  188  190  201  202  205  216  229  261  272
  328  335  342  351  362  364  378  388  393  402  408  416  442  443
  452  455  489  496  503  512  516  517  518  519  520  523  525  526
  530  531  532  533  534  535  537  538  540  541  542  551  554  558
  561  562  563  566  569  570  572  574  576  581  583  586  587  590
  592  593  595  598  599  602  603  604  605  608  615  616  617  618
  619  623  625  626  628  630  636  641  643  646  648  650  653  654
  655  657  663  665  666  667  671  674  675  681  682  683  684  685
  687  688  690  693  696  697  702  703  704  707  708  709  711  712
  714  719  720  722  723  725  730  731  734  736  738  740  742  750
  751  755  760  762  764  765  768  769  770  771  773  774  775  776
  777  778  779  780  781  782  783  786  787  788  789  790  792  793
  794  795  796  797  798  799  800  802  803  804  805  810 